In [ ]:
import sys
sys.path.append('..')

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage.filters import gaussian, threshold_otsu
from skimage.transform import resize
from skimage import io
import pickle
from skimage.measure import regionprops

import matplotlib.pyplot as plt
import torch
import time
import model.model_cnnvae_conditional
import model.optimizer as optimizer
from sklearn.decomposition import PCA
import pandas as pd

In [ ]:
sharedSizes=[1024]
dSpecific_filter=[(200,16)]
pID_type='randInit'
pIDemb_size=64

In [ ]:
holdOutSamples=['HV1','P22','P14','P27']

In [ ]:
sourceDir='/data/xinyi/c2p/data/chromark'
segDir=os.path.join(sourceDir,'nuclear_masks')
imgDir=os.path.join(sourceDir,'raw_data')
conditions=['controls','headneck','meningioma', 'glioma']

outSize=128
savename='pathCentered_'+str(outSize)

imgsC_all=None
imgsP_all=None
imgNames_all=None
proteinNames=None
pID_all=None
for condition_i in conditions:
    print(condition_i)
    segDir_i=os.path.join(segDir,condition_i)
    imgDir_i=os.path.join(imgDir,condition_i)
    for stain in os.listdir(segDir_i):
        print(stain)
        segDir_i_stain=os.path.join(segDir_i,stain)
        imgDir_i_stain=os.path.join(imgDir_i,stain)
        
        segPID2name={}
        for pID_dir in os.listdir(segDir_i_stain):
            pID=pID_dir.split('_')
            segPID2name[pID[0]]=pID_dir
        imgPID2name={}
        for pID_dir in os.listdir(imgDir_i_stain):
            pID=pID_dir.split('_')
            imgPID2name[pID[0]]=pID_dir
        for pID in segPID2name.keys():
            if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                continue
            if pID in holdOutSamples:
                print('hold out: '+pID)
                continue
            print(pID)
            if pID not in imgPID2name:
                print('img not found '+pID)
                continue
            imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
            segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])
            
            with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                imgNames=pickle.load(output)
            with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                img=pickle.load(output)

            imgP=np.zeros((img.shape[0],1,img.shape[2],img.shape[3]))
            proteinNames_curr=np.array([])
            stain_list=stain.split('_')
            nImgPerStain=int(img.shape[0]/(len(stain_list)-1))
            np.random.seed(3)
            allIdx_all=np.arange(img.shape[0])
            np.random.shuffle(allIdx_all)
            for s in range(1,len(stain_list)):
                s_start=(s-1)*nImgPerStain
                if s==len(stain_list)-1:
                    s_end=img.shape[0]
                else:
                    s_end=s*nImgPerStain
                imgP[s_start:s_end]=img[allIdx_all[s_start:s_end],s].reshape(s_end-s_start,1,img.shape[2],img.shape[3])
                proteinNames_curr=np.concatenate((proteinNames_curr,np.repeat(stain_list[s],s_end-s_start)))
            
            if pID_all is None:
                pID_all=np.repeat(pID,img.shape[0])
                imgsC_all=img[allIdx_all,[0]]
                imgNames_all=imgNames[allIdx_all]
                proteinNames=proteinNames_curr
                imgsP_all=imgP
            else:
                pID_all=np.concatenate((pID_all,np.repeat(pID,img.shape[0])))
                imgsC_all=np.concatenate((imgsC_all,img[allIdx_all,[0]]),axis=0)
                imgNames_all=np.concatenate((imgNames_all,imgNames[allIdx_all]))
                proteinNames=np.concatenate((proteinNames,proteinNames_curr))
                imgsP_all=np.concatenate((imgsP_all,imgP),axis=0)
imgsC_all=imgsC_all.reshape(imgsC_all.shape[0],1,imgsC_all.shape[1],imgsC_all.shape[2])

In [ ]:
imgsC_val=None
imgsP_val=None
imgNames_val=None
proteinNames_val=None
pID_val=None
imgsP_val_all=None
imgsP_val_all_names=None
for condition_i in conditions:
    print(condition_i)
    segDir_i=os.path.join(segDir,condition_i)
    imgDir_i=os.path.join(imgDir,condition_i)
    for stain in os.listdir(segDir_i):
        print(stain)
        segDir_i_stain=os.path.join(segDir_i,stain)
        imgDir_i_stain=os.path.join(imgDir_i,stain)
        
        segPID2name={}
        for pID_dir in os.listdir(segDir_i_stain):
            pID=pID_dir.split('_')
            segPID2name[pID[0]]=pID_dir
        imgPID2name={}
        for pID_dir in os.listdir(imgDir_i_stain):
            pID=pID_dir.split('_')
            imgPID2name[pID[0]]=pID_dir
        for pID in segPID2name.keys():
            if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                continue
            if pID not in holdOutSamples:
                continue
            print(pID)
            if pID not in imgPID2name:
                print('img not found '+pID)
                continue
            imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
            segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])
            
            with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                imgNames=pickle.load(output)
            with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                img=pickle.load(output)

            imgP=np.zeros((img.shape[0],1,img.shape[2],img.shape[3]))
            imgP_all=np.zeros((img.shape[0],3,img.shape[2],img.shape[3]))
            proteinNames_val_curr=np.array([])
            imgsP_val_all_names_curr=None
            stain_list=stain.split('_')
            nImgPerStain=int(img.shape[0]/(len(stain_list)-1))
            np.random.seed(3)
            allIdx_all=np.arange(img.shape[0])
            np.random.shuffle(allIdx_all)
            for s in range(1,len(stain_list)):
                s_start=(s-1)*nImgPerStain
                if s==len(stain_list)-1:
                    s_end=img.shape[0]
                else:
                    s_end=s*nImgPerStain
                imgP[s_start:s_end]=img[allIdx_all[s_start:s_end],s].reshape(s_end-s_start,1,img.shape[2],img.shape[3])
                proteinNames_val_curr=np.concatenate((proteinNames_val_curr,np.repeat(stain_list[s],s_end-s_start)))
                imgP_all[s_start:s_end,:img.shape[1]-1]=img[allIdx_all[s_start:s_end],1:].reshape(s_end-s_start,img.shape[1]-1,img.shape[2],img.shape[3])
                if imgsP_val_all_names_curr is None:
                    imgsP_val_all_names_curr=np.tile(stain_list[1:],(s_end-s_start,1))
                else:
                    imgsP_val_all_names_curr=np.concatenate((imgsP_val_all_names_curr,np.tile(stain_list[1:],(s_end-s_start,1))),axis=0)
            if imgsP_val_all_names_curr.shape[1]==2:
                imgsP_val_all_names_curr=np.hstack((imgsP_val_all_names_curr,np.repeat('None',imgsP_val_all_names_curr.shape[0]).reshape(-1,1)))
            if pID_val is None:
                pID_val=np.repeat(pID,img.shape[0])
                imgsC_val=img[allIdx_all,[0]]
                imgNames_val=imgNames[allIdx_all]
                proteinNames_val=proteinNames_val_curr
                imgsP_val=imgP
                imgsP_val_all=imgP_all
                imgsP_val_all_names=imgsP_val_all_names_curr
            else:
                pID_val=np.concatenate((pID_val,np.repeat(pID,img.shape[0])))
                imgsC_val=np.concatenate((imgsC_val,img[allIdx_all,[0]]),axis=0)
                imgNames_val=np.concatenate((imgNames_val,imgNames[allIdx_all]))
                proteinNames_val=np.concatenate((proteinNames_val,proteinNames_val_curr))
                imgsP_val=np.concatenate((imgsP_val,imgP),axis=0)
                imgsP_val_all=np.concatenate((imgsP_val_all,imgP_all),axis=0)
                imgsP_val_all_names=np.concatenate((imgsP_val_all_names,imgsP_val_all_names_curr),axis=0)
imgsC_val=imgsC_val.reshape(imgsC_val.shape[0],1,imgsC_val.shape[1],imgsC_val.shape[2])

In [ ]:
nProt=np.unique(proteinNames).size
pnames,revIdx,pCounts=np.unique(proteinNames,return_inverse=True,return_counts=True)
plabels=torch.tensor(np.arange(pnames.size)[revIdx]).long()

In [ ]:
nProt_val=np.unique(proteinNames_val).size
pnames_val,revIdx_val,pCounts_val=np.unique(proteinNames_val,return_inverse=True,return_counts=True)
plabels_val=torch.tensor(np.arange(pnames_val.size)[revIdx_val]).long()

In [ ]:
nProt==nProt_val

In [ ]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "6"

In [ ]:
#VAE settings
seed=3
epochs=5001
saveFreq=100
lr=0.001 #initial learning rate
weight_decay=0 #Weight for L2 loss on embedding matrix.

# batchsize=4
batchsize=256
kernel_size=4
stride=2
padding=1

# fc_dim1=6000
hidden1=64 #Number of channels in hidden layer 1
hidden2=128 
hidden3=256
hidden4=256
hidden5=96
hidden5_xy=4
fc_dim1=96*hidden5_xy*hidden5_xy
fc_dim2=6000

dropout=0.01
kl_weight=0.0000001





In [ ]:
trainIdx=np.arange(imgsC_all.shape[0])
printFreq=1
valIdx=np.arange(imgsC_val.shape[0])
def train(epoch):
    t = time.time()
    modelcnn_dna.train()
    modelcnn_protein.train()

    loss_x_train_all_dna=0
    loss_x_trainShared_all_dna=0
    loss_x_train_all_protein=0
    loss_x_trainShared_all_protein=0
    loss_all=0
    ntrainBatches=int(np.ceil(trainIdx.shape[0]/batchsize))
    for i in range(ntrainBatches):
#         if i%200==0:
#         print(i)
        trainIdx_i=trainIdx[i*batchsize:min((i+1)*batchsize,trainIdx.shape[0])]
        traintarget_protein=torch.tensor(imgsP_all[trainIdx_i]).cuda().float()
        trainIdx_i=torch.tensor(trainIdx_i)
        traintarget_dna=torch.tensor(imgsC_all[trainIdx_i]).cuda().float()
        trainInput_ID=plabels[trainIdx_i].cuda()
        trainInput_shared=latent_shared_dec(trainIdx_i).cuda().float()
        trainInput_dna=latent_dna_dec(trainIdx_i).cuda().float()
        trainInput_protein=latent_protein_dec(trainIdx_i).cuda().float()
#         print(trainInput.shape)

        optimizer_dna.zero_grad()
        optimizer_protein.zero_grad()

        reconShared_dna,recon_dna= modelcnn_dna(traintarget_dna,trainInput_ID)
        reconShared_protein,recon_protein= modelcnn_protein(traintarget_protein,trainInput_ID)
        
        loss_x_train_protein=loss_x(recon_protein, trainInput_protein)
        loss_xShared_train_protein=loss_x(reconShared_protein,trainInput_shared)
        loss_x_train_dna=loss_x(recon_dna, trainInput_dna)
        loss_xShared_train_dna=loss_x(reconShared_dna,trainInput_shared)
        
        loss=loss_x_train_protein+loss_xShared_train_protein+loss_x_train_dna+loss_xShared_train_dna
        
        loss_x_train_all_dna+=loss_x_train_dna.item()
        loss_x_trainShared_all_dna+=loss_xShared_train_dna.item()
        loss_x_train_all_protein+=loss_x_train_protein.item()
        loss_x_trainShared_all_protein+=loss_xShared_train_protein.item()
        loss_all+=loss.item()
        
        loss.backward()
        optimizer_dna.step()
        optimizer_protein.step()


    loss_x_train_all_dna=loss_x_train_all_dna/ntrainBatches
    loss_x_trainShared_all_dna=loss_x_trainShared_all_dna/ntrainBatches
    loss_x_train_all_protein=loss_x_train_all_protein/ntrainBatches
    loss_x_trainShared_all_protein=loss_x_trainShared_all_protein/ntrainBatches
    loss_all=loss_all/ntrainBatches

    with torch.no_grad():
        modelcnn_dna.eval()
        modelcnn_protein.eval()



        loss_val_all=0
        loss_x_val_all_dna=0
        loss_x_valShared_all_dna=0
        loss_x_val_all_protein=0
        loss_x_valShared_all_protein=0
        nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
        for i in range(nvalBatches):
            valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
            valtarget_protein=torch.tensor(imgsP_val[valIdx_i]).cuda().float()
            valtarget_dna=torch.tensor(imgsC_val[valIdx_i]).cuda().float()
            valInput_ID=plabels_val[valIdx_i].cuda()
            valIdx_i=torch.tensor(valIdx_i)

            reconShared_dna_l,recon_dna_l= modelcnn_dna(valtarget_dna,valInput_ID)
            reconShared_protein_l,recon_protein_l= modelcnn_protein(valtarget_protein,valInput_ID)
            
            recon_dna= modelcnn_dna_dec(torch.cat((reconShared_dna_l,recon_dna_l),dim=1),pIDemb_dec(valInput_ID))
            reconShared_dna=modelcnn_dnaShared_dec(reconShared_dna_l,pIDemb_dec(valInput_ID))
            recon_protein= modelcnn_protein_dec(torch.cat((reconShared_protein_l,recon_protein_l),dim=1),pIDemb_dec(valInput_ID))
            reconShared_protein=modelcnn_pShared_dec(reconShared_protein_l,pIDemb_dec(valInput_ID))


            loss_x_val_protein=loss_imgRecon(recon_protein, valtarget_protein)
            loss_xShared_val_protein=loss_imgRecon(reconShared_protein,valtarget_protein)
            loss_x_val_dna=loss_imgRecon(recon_dna, valtarget_dna)
            loss_xShared_val_dna=loss_imgRecon(reconShared_dna,valtarget_dna)

            loss=loss_x_val_protein+loss_xShared_val_protein+loss_x_val_dna+loss_xShared_val_dna

            loss_x_val_all_dna+=loss_x_val_dna.item()
            loss_x_valShared_all_dna+=loss_xShared_val_dna.item()
            loss_x_val_all_protein+=loss_x_val_protein.item()
            loss_x_valShared_all_protein+=loss_xShared_val_protein.item()
            loss_val_all+=loss.item()

        loss_x_val_all_dna=loss_x_val_all_dna/nvalBatches
        loss_x_valShared_all_dna=loss_x_valShared_all_dna/nvalBatches
        loss_x_val_all_protein=loss_x_val_all_protein/nvalBatches
        loss_x_valShared_all_protein=loss_x_valShared_all_protein/nvalBatches
        loss_val_all=loss_val_all/nvalBatches



    if epoch%printFreq==0:
        print('Epoch: {:04d}'.format(epoch),
              'loss_train: {:.4f}'.format(loss_all),
              'loss_x_train_dna: {:.4f}'.format(loss_x_train_all_dna),
              'loss_x_train_protein: {:.4f}'.format(loss_x_train_all_protein),
              'loss_xShared_train_dna: {:.4f}'.format(loss_x_trainShared_all_dna),
              'loss_xShared_train_protein: {:.4f}'.format(loss_x_trainShared_all_protein),
              'loss_x_val_dna: {:.4f}'.format(loss_x_val_all_dna),
              'loss_x_val_protein: {:.4f}'.format(loss_x_val_all_protein),
              'loss_xShared_val_dna: {:.4f}'.format(loss_x_valShared_all_dna),
              'loss_xShared_val_protein: {:.4f}'.format(loss_x_valShared_all_protein),
              'time: {:.4f}s'.format(time.time() - t))
    return loss_all,loss_x_train_all_dna,loss_x_train_all_protein,loss_x_trainShared_all_dna,loss_x_trainShared_all_protein,loss_val_all,loss_x_val_all_dna,loss_x_val_all_protein,loss_x_valShared_all_dna,loss_x_valShared_all_protein

In [ ]:
match_weight=1
sharedWeight=1
name_train='splitChannels_conditional_lord_withNoise_bce'
modelname_train='cnn_vae_pbmc_lord'
logsavepath_train=os.path.join('/data/xinyi/c2p/log/',modelname_train,name_train)
modelsavepath_train=os.path.join('/data/xinyi/c2p/models/',modelname_train,name_train)
plotsavepath_train=os.path.join('/data/xinyi/c2p/plots/',modelname_train,name_train)

if not os.path.exists(os.path.join('/data/xinyi/c2p/log/',modelname_train)):
    os.mkdir(os.path.join('/data/xinyi/c2p/log/',modelname_train))
    os.mkdir(os.path.join('/data/xinyi/c2p/models/',modelname_train))
    os.mkdir(os.path.join('/data/xinyi/c2p/plots/',modelname_train))
if not os.path.exists(logsavepath_train):
    os.mkdir(logsavepath_train)
    os.mkdir(os.path.join(logsavepath_train,'dna'))
    os.mkdir(os.path.join(logsavepath_train,'protein'))
if not os.path.exists(modelsavepath_train):
    os.mkdir(modelsavepath_train)
    os.mkdir(os.path.join(modelsavepath_train,'dna'))
    os.mkdir(os.path.join(modelsavepath_train,'protein'))
if not os.path.exists(plotsavepath_train):
    os.mkdir(plotsavepath_train)
    os.mkdir(os.path.join(plotsavepath_train,'dna'))
    os.mkdir(os.path.join(plotsavepath_train,'protein'))


loadEpoch='3399'


In [ ]:

logsavepath_p_dna=os.path.join(logsavepath_train,'dna')
modelsavepath_p_dna=os.path.join(modelsavepath_train,'dna')
plotsavepath_p_dna=os.path.join(plotsavepath_train,'dna')
if not os.path.exists(logsavepath_p_dna):
    os.mkdir(logsavepath_p_dna)
if not os.path.exists(modelsavepath_p_dna):
    os.mkdir(modelsavepath_p_dna)
if not os.path.exists(plotsavepath_p_dna):
    os.mkdir(plotsavepath_p_dna)

logsavepath_p_protein=os.path.join(logsavepath_train,'protein')
modelsavepath_p_protein=os.path.join(modelsavepath_train,'protein')
plotsavepath_p_protein=os.path.join(plotsavepath_train,'protein')
if not os.path.exists(logsavepath_p_protein):
    os.mkdir(logsavepath_p_protein)
if not os.path.exists(modelsavepath_p_protein):
    os.mkdir(modelsavepath_p_protein)
if not os.path.exists(plotsavepath_p_protein):
    os.mkdir(plotsavepath_p_protein)

#train-test split
np.random.seed(3)
pctVal=0.05
pctTest=0.1

allIdx_all=np.arange(proteinNames.size)
np.random.shuffle(allIdx_all)
valIdx_all=allIdx_all[:int(pctVal*proteinNames.size)]
testIdx_all=allIdx_all[int(pctVal*proteinNames.size):(int(pctVal*proteinNames.size)+int(pctTest*proteinNames.size))]
trainIdx_all=allIdx_all[(int(pctVal*proteinNames.size)+int(pctTest*proteinNames.size)):]



for currLatentSize in sharedSizes:
    for dSpecificSize,dfilterSize in dSpecific_filter:
        latent_curr=None
#         if sIdx==0:
#             continue


        print(currLatentSize)
        print(dSpecificSize)
        dna_cShared=hidden5-dfilterSize
        p_cShared=dna_cShared

        loss_kl=optimizer.optimizer_kl
        loss_x=torch.nn.MSELoss()
        loss_imgRecon=torch.nn.BCEWithLogitsLoss()

        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.enabled = True
        if modelname_train=='cnn_vae_pbmc_lord':
            modelcnn_dna_dec = model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize+dSpecificSize,pIDemb_size,applySigmoid=False)
            modelcnn_protein_dec = model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize+dSpecificSize,pIDemb_size,applySigmoid=False)
            modelcnn_dnaShared_dec=model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize,pIDemb_size,applySigmoid=False)
            modelcnn_pShared_dec=model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize,pIDemb_size,applySigmoid=False)
        modelcnn_dna_dec.cuda().eval()
        modelcnn_protein_dec.cuda().eval()
        modelcnn_dnaShared_dec.cuda().eval()
        modelcnn_pShared_dec.cuda().eval()
        
        batchsize=328
        latent_shared_dec=torch.nn.Embedding(proteinNames.size,currLatentSize)
        
        latent_protein_dec=torch.nn.Embedding(proteinNames.size,dSpecificSize)
        latent_dna_dec=torch.nn.Embedding(proteinNames.size,dSpecificSize)
        pIDemb_dec=torch.nn.Embedding(nProt, pIDemb_size).cuda()

        
        if loadEpoch is not None:
            with open(os.path.join(modelsavepath_p_dna,'latent_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
                latent_dna_dec.weight=pickle.load(output)
            with open(os.path.join(modelsavepath_p_dna,'latentShared_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
                latent_shared_dec.weight=pickle.load(output)
            with open(os.path.join(modelsavepath_p_protein,'latent_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
                latent_protein_dec.weight=pickle.load(output)
            with open(os.path.join(modelsavepath_p_protein,'pIDemb_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
                pIDemb_dec.weight=pickle.load(output)
            
            with open(os.path.join(modelsavepath_p_dna,'stateDict_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
                stateDict_train_dna_dec=pickle.load(output)
            with open(os.path.join(modelsavepath_p_protein,'stateDict_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
                stateDict_train_protein_dec=pickle.load(output)
            with open(os.path.join(modelsavepath_p_dna,'stateDictShared_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
                stateDict_train_dnaShared_dec=pickle.load(output)
            with open(os.path.join(modelsavepath_p_protein,'stateDictShared_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
                stateDict_train_proteinShared_dec=pickle.load(output)

            modelcnn_dna_dec.load_state_dict(stateDict_train_dna_dec[int(loadEpoch)])
            modelcnn_dnaShared_dec.load_state_dict(stateDict_train_dnaShared_dec[int(loadEpoch)])
            modelcnn_protein_dec.load_state_dict(stateDict_train_protein_dec[int(loadEpoch)])
            modelcnn_pShared_dec.load_state_dict(stateDict_train_proteinShared_dec[int(loadEpoch)])
        latent_shared_dec.weight.requires_grad=False
        
        latent_protein_dec.weight.requires_grad=False
        latent_dna_dec.weight.requires_grad=False
        pIDemb_dec.weight.requires_grad=False



        train_loss=[np.inf]*(epochs)
        train_loss_x_dna=[np.inf]*(epochs)
        train_loss_xShared_dna=[np.inf]*(epochs)
        val_loss=[np.inf]*(epochs)
        val_loss_x_dna=[np.inf]*(epochs)
        val_loss_xShared_dna=[np.inf]*(epochs)

        train_loss_x_protein=[np.inf]*(epochs)
        train_loss_xShared_protein=[np.inf]*(epochs)
        val_loss_x_protein=[np.inf]*(epochs)
        val_loss_xShared_protein=[np.inf]*(epochs)

        t_ep=time.time()

        stateDict_train_dna={}
        stateDict_train_protein={}
        stateDict_train_dnaShared={}
        stateDict_train_proteinShared={}
        
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.enabled = True
        if modelname_train=='cnn_vae_pbmc_lord':
            modelcnn_dna = model.model_cnnvae_conditional.CNN_VAE_split_encode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5,p_cShared,dna_cShared*hidden5_xy*hidden5_xy, (hidden5-p_cShared)*hidden5_xy*hidden5_xy,currLatentSize,dSpecificSize,pnames.size,'randInit',pIDemb_size)
            modelcnn_protein = model.model_cnnvae_conditional.CNN_VAE_split_encode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5,p_cShared,p_cShared*hidden5_xy*hidden5_xy, (hidden5-p_cShared)*hidden5_xy*hidden5_xy,currLatentSize,dSpecificSize,pnames.size,'randInit',pIDemb_size)
        modelcnn_dna.cuda()
        modelcnn_protein.cuda()
        
        optimizer_dna = torch.optim.Adam(modelcnn_dna.parameters(), lr=lr, weight_decay=weight_decay)
        optimizer_protein = torch.optim.Adam(modelcnn_protein.parameters(), lr=lr, weight_decay=weight_decay)


        
        
                
        epCounts=0
        for ep in range(epochs):
            train_loss[ep],train_loss_x_dna[ep],train_loss_x_protein[ep],train_loss_xShared_dna[ep],train_loss_xShared_protein[ep],val_loss[ep],val_loss_x_dna[ep],val_loss_x_protein[ep],val_loss_xShared_dna[ep],val_loss_xShared_protein[ep]=train(ep)

            if ep>200 and (val_loss_x_dna[ep]>=val_loss_x_dna[ep-200] or val_loss_x_protein[ep]>=val_loss_x_protein[ep-200]):
                epCounts+=1
            else:
                epCounts=0

            if epCounts>100:
                break


            if ep%saveFreq == (saveFreq-1):
                stateDict_train_dna[ep]=modelcnn_dna.cpu().state_dict()
                stateDict_train_protein[ep]=modelcnn_protein.cpu().state_dict()

            modelcnn_dna.cuda()
            modelcnn_protein.cuda()
            torch.cuda.empty_cache()
        print(' total time: {:.4f}s'.format(time.time() - t_ep))

        with open(os.path.join(modelsavepath_p_dna,'stateDict_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(stateDict_train_dna, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(modelsavepath_p_protein,'stateDict_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(stateDict_train_protein, output, pickle.HIGHEST_PROTOCOL)


        with open(os.path.join(logsavepath_p_dna,'train_loss_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(train_loss, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(logsavepath_p_dna,'train_loss_x_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(train_loss_x_dna, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(logsavepath_p_dna,'train_loss_xShared_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(train_loss_xShared_dna, output, pickle.HIGHEST_PROTOCOL)

        with open(os.path.join(logsavepath_p_dna,'val_loss_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(val_loss, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(logsavepath_p_dna,'val_loss_x_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(val_loss_x_dna, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(logsavepath_p_dna,'val_loss_xShared_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(val_loss_xShared_dna, output, pickle.HIGHEST_PROTOCOL)

        with open(os.path.join(logsavepath_p_protein,'train_loss_x_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(train_loss_x_protein, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(logsavepath_p_protein,'val_loss_x_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(val_loss_x_protein, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(logsavepath_p_protein,'train_loss_xShared_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(train_loss_xShared_protein, output, pickle.HIGHEST_PROTOCOL)
        with open(os.path.join(logsavepath_p_protein,'val_loss_xShared_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
            pickle.dump(val_loss_xShared_protein, output, pickle.HIGHEST_PROTOCOL)

        totalepoch=np.argmin(np.array(val_loss_x_dna)+np.array(val_loss_x_protein))

        print('loss_val_p: {:.4f}'.format(val_loss_x_protein[totalepoch]),
              'loss_val_c: {:.4f}'.format(val_loss_x_dna[totalepoch]))


        plt.plot(np.arange(epochs),train_loss_x_dna)
        plt.plot(np.arange(epochs),val_loss_x_dna)
        # plt.plot(np.arange(epochs),train_loss_kl_ep)
        plt.legend(['training x recon loss','validation x recon loss','training kl loss'],loc='upper right')
        plt.savefig(os.path.join(plotsavepath_p_dna,'loss_seed3_x_encode_v1'+str(currLatentSize)+'_'+str(dSpecificSize)+'.jpg'))
        plt.show()


        plt.plot(np.arange(epochs),train_loss_x_protein)
        plt.plot(np.arange(epochs),val_loss_x_protein)
        # plt.plot(np.arange(epochs),train_loss_kl_ep)
        plt.legend(['training x recon loss','validation x recon loss','training kl loss'],loc='upper right')
        plt.savefig(os.path.join(plotsavepath_p_protein,'loss_seed3_x_encode_v1'+str(currLatentSize)+'_'+str(dSpecificSize)+'.jpg'))
        plt.show()

        plt.plot(np.arange(epochs),train_loss_xShared_dna)
        plt.plot(np.arange(epochs),val_loss_xShared_dna)
        # plt.plot(np.arange(epochs),train_loss_kl_ep)
        plt.legend(['training shared recon','validation shared recon'],loc='upper right')
        plt.savefig(os.path.join(plotsavepath_p_dna,'loss_seed3_xShared_encode_v1'+str(currLatentSize)+'_'+str(dSpecificSize)+'.jpg'))
        plt.show()

        plt.plot(np.arange(epochs),train_loss_xShared_protein)
        plt.plot(np.arange(epochs),val_loss_xShared_protein)
        # plt.plot(np.arange(epochs),train_loss_kl_ep)
        plt.legend(['training shared recon','validation shared recon'],loc='upper right')
        plt.savefig(os.path.join(plotsavepath_p_protein,'loss_seed3_xShared_encode_v1'+str(currLatentSize)+'_'+str(dSpecificSize)+'.jpg'))
        plt.show()










In [ ]:
stateDict_train_dna[ep]=modelcnn_dna.cpu().state_dict()
stateDict_train_protein[ep]=modelcnn_protein.cpu().state_dict()

In [ ]:
with open(os.path.join(modelsavepath_p_dna,'stateDict_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
    pickle.dump(stateDict_train_dna, output, pickle.HIGHEST_PROTOCOL)
with open(os.path.join(modelsavepath_p_protein,'stateDict_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'wb') as output:
    pickle.dump(stateDict_train_protein, output, pickle.HIGHEST_PROTOCOL)



In [ ]:
plt.plot(np.arange(epochs),val_loss_x_dna)
# plt.plot(np.arange(epochs),train_loss_kl_ep)
plt.legend(['validation x recon loss','training kl loss'],loc='upper right')
plt.savefig(os.path.join(plotsavepath_p_dna,'loss_seed3_xval_encode_v1'+str(currLatentSize)+'_'+str(dSpecificSize)+'.jpg'))
plt.show()


plt.plot(np.arange(epochs),val_loss_x_protein)
# plt.plot(np.arange(epochs),train_loss_kl_ep)
plt.legend(['validation x recon loss','training kl loss'],loc='upper right')
plt.savefig(os.path.join(plotsavepath_p_protein,'loss_seed3_xval_encode_v1'+str(currLatentSize)+'_'+str(dSpecificSize)+'.jpg'))
plt.show()

plt.plot(np.arange(epochs),val_loss_xShared_dna)
# plt.plot(np.arange(epochs),train_loss_kl_ep)
plt.legend(['validation shared recon'],loc='upper right')
plt.savefig(os.path.join(plotsavepath_p_dna,'loss_seed3_xSharedval_encode_v1'+str(currLatentSize)+'_'+str(dSpecificSize)+'.jpg'))
plt.show()

plt.plot(np.arange(epochs),val_loss_xShared_protein)
# plt.plot(np.arange(epochs),train_loss_kl_ep)
plt.legend(['validation shared recon'],loc='upper right')
plt.savefig(os.path.join(plotsavepath_p_protein,'loss_seed3_xSharedval_encode_v1'+str(currLatentSize)+'_'+str(dSpecificSize)+'.jpg'))
plt.show()

In [ ]:
logsavepath_p_dna=os.path.join(logsavepath_train,'dna')
modelsavepath_p_dna=os.path.join(modelsavepath_train,'dna')
plotsavepath_p_dna=os.path.join(plotsavepath_train,'dna')

logsavepath_p_protein=os.path.join(logsavepath_train,'protein')
modelsavepath_p_protein=os.path.join(modelsavepath_train,'protein')
plotsavepath_p_protein=os.path.join(plotsavepath_train,'protein')
#train-test split
np.random.seed(3)
pctVal=0.05
pctTest=0.1

allIdx_all=np.arange(proteinNames.size)
np.random.shuffle(allIdx_all)
valIdx_all=allIdx_all[:int(pctVal*proteinNames.size)]
testIdx_all=allIdx_all[int(pctVal*proteinNames.size):(int(pctVal*proteinNames.size)+int(pctTest*proteinNames.size))]
trainIdx_all=allIdx_all[(int(pctVal*proteinNames.size)+int(pctTest*proteinNames.size)):]



for currLatentSize in sharedSizes:
    for dSpecificSize,dfilterSize in dSpecific_filter:
        latent_curr=None
#         if sIdx==0:
#             continue


        print(currLatentSize)
        print(dSpecificSize)
        dna_cShared=hidden5-dfilterSize
        p_cShared=dna_cShared

        loss_kl=optimizer.optimizer_kl
        loss_x=torch.nn.MSELoss()

        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.enabled = True
        if modelname_train=='cnn_vae_pbmc_lord':
            modelcnn_dna_dec = model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize+dSpecificSize,pIDemb_size,applySigmoid=False)
            modelcnn_protein_dec = model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize+dSpecificSize,pIDemb_size,applySigmoid=False)
            modelcnn_dnaShared_dec=model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize,pIDemb_size,applySigmoid=False)
            modelcnn_pShared_dec=model.model_cnnvae_conditional.CNN_VAE_decode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5, fc_dim1,currLatentSize,pIDemb_size,applySigmoid=False)
        modelcnn_dna_dec.cuda().eval()
        modelcnn_protein_dec.cuda().eval()
        modelcnn_dnaShared_dec.cuda().eval()
        modelcnn_pShared_dec.cuda().eval()
        
        batchsize=328
        latent_shared_dec=torch.nn.Embedding(proteinNames.size,currLatentSize)
        
        latent_protein_dec=torch.nn.Embedding(proteinNames.size,dSpecificSize)
        latent_dna_dec=torch.nn.Embedding(proteinNames.size,dSpecificSize)
        pIDemb_dec=torch.nn.Embedding(nProt, pIDemb_size).cuda()

        
        if loadEpoch is not None:
            with open(os.path.join(modelsavepath_p_dna,'latent_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
                latent_dna_dec.weight=pickle.load(output)
            with open(os.path.join(modelsavepath_p_dna,'latentShared_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
                latent_shared_dec.weight=pickle.load(output)
            with open(os.path.join(modelsavepath_p_protein,'latent_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
                latent_protein_dec.weight=pickle.load(output)
            with open(os.path.join(modelsavepath_p_protein,'pIDemb_'+str(currLatentSize)+'_'+str(dSpecificSize)+'_'+str(loadEpoch)), 'rb') as output:
                pIDemb_dec.weight=pickle.load(output)
            
            with open(os.path.join(modelsavepath_p_dna,'stateDict_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
                stateDict_train_dna_dec=pickle.load(output)
            with open(os.path.join(modelsavepath_p_protein,'stateDict_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
                stateDict_train_protein_dec=pickle.load(output)
            with open(os.path.join(modelsavepath_p_dna,'stateDictShared_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
                stateDict_train_dnaShared_dec=pickle.load(output)
            with open(os.path.join(modelsavepath_p_protein,'stateDictShared_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
                stateDict_train_proteinShared_dec=pickle.load(output)

            modelcnn_dna_dec.load_state_dict(stateDict_train_dna_dec[int(loadEpoch)])
            modelcnn_dnaShared_dec.load_state_dict(stateDict_train_dnaShared_dec[int(loadEpoch)])
            modelcnn_protein_dec.load_state_dict(stateDict_train_protein_dec[int(loadEpoch)])
            modelcnn_pShared_dec.load_state_dict(stateDict_train_proteinShared_dec[int(loadEpoch)])
        latent_shared_dec.weight.requires_grad=False
        
        latent_protein_dec.weight.requires_grad=False
        latent_dna_dec.weight.requires_grad=False
        pIDemb_dec.weight.requires_grad=False



        train_loss=[np.inf]*(epochs)
        train_loss_x_dna=[np.inf]*(epochs)
        train_loss_xShared_dna=[np.inf]*(epochs)
        val_loss=[np.inf]*(epochs)
        val_loss_x_dna=[np.inf]*(epochs)
        val_loss_xShared_dna=[np.inf]*(epochs)

        train_loss_x_protein=[np.inf]*(epochs)
        train_loss_xShared_protein=[np.inf]*(epochs)
        val_loss_x_protein=[np.inf]*(epochs)
        val_loss_xShared_protein=[np.inf]*(epochs)

        t_ep=time.time()

        stateDict_train_dna={}
        stateDict_train_protein={}
        stateDict_train_dnaShared={}
        stateDict_train_proteinShared={}
        
        np.random.seed(seed)
        torch.manual_seed(seed)
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.enabled = True
        if modelname_train=='cnn_vae_pbmc_lord':
            modelcnn_dna = model.model_cnnvae_conditional.CNN_VAE_split_encode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5,p_cShared,dna_cShared*hidden5_xy*hidden5_xy, (hidden5-p_cShared)*hidden5_xy*hidden5_xy,currLatentSize,dSpecificSize,pnames.size,'randInit',pIDemb_size)
            modelcnn_protein = model.model_cnnvae_conditional.CNN_VAE_split_encode_pIDemb(kernel_size, stride, padding, 1, hidden1, hidden2, hidden3, hidden4, hidden5,p_cShared,p_cShared*hidden5_xy*hidden5_xy, (hidden5-p_cShared)*hidden5_xy*hidden5_xy,currLatentSize,dSpecificSize,pnames.size,'randInit',pIDemb_size)
        modelcnn_dna.cuda()
        modelcnn_protein.cuda()

In [ ]:
with open(os.path.join(modelsavepath_p_dna,'stateDict_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
    stateDict_train_dna=pickle.load( output)
with open(os.path.join(modelsavepath_p_protein,'stateDict_encode_v1_'+str(currLatentSize)+'_'+str(dSpecificSize)), 'rb') as output:
    stateDict_train_protein=pickle.load(output)


In [ ]:
stateDict_train_dna.keys()

In [ ]:
modelcnn_dna.load_state_dict(stateDict_train_dna[399])
modelcnn_protein.load_state_dict(stateDict_train_protein[399])

In [ ]:
def percentileNorm(img_c):
#     intensity,intCounts=np.unique(img_c,return_counts=True)
#     modeint=intensity[np.argmax(intCounts)]
    modeint=np.percentile(img_c,80)
    img_c=img_c-modeint
    img_c[img_c<0]=0
    img_c=img_c/np.max(img_c)
    print(modeint)
    return img_c

def modeSub(img_c):
    intensity,intCounts=np.unique(img_c,return_counts=True)
    modeint=intensity[np.argmax(intCounts)]
#     modeint=np.percentile(img_c,75)
    img_c=img_c-modeint
    img_c[img_c<0]=0
    img_c=img_c/np.max(img_c)
    print(modeint)
    return img_c

def modeSub_torch(img_c):
    intensity,intCounts=torch.unique(img_c,return_counts=True)
    modeint=intensity[torch.argmax(intCounts)]
#     modeint=np.percentile(img_c,75)
    img_c=img_c-modeint
    img_c[img_c<0]=0
    img_c=img_c/torch.max(img_c)
#     print(modeint)
    return img_c

In [ ]:
sigmoid=torch.nn.Sigmoid()

In [ ]:
#plotting subtraction of mode intensity
with torch.no_grad():
    modelcnn_dna.eval()
    modelcnn_protein.eval()
    modelcnn_dna.cuda()
    modelcnn_protein.cuda()



    loss_val_all=0
    loss_x_val_all_dna=0
    loss_x_valShared_all_dna=0
    loss_x_val_all_protein=0
    loss_x_valShared_all_protein=0
    nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
    for i in range(nvalBatches):
        valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
        valtarget_protein=torch.tensor(imgsP_val[valIdx_i]).cuda().float()
        valtarget_dna=torch.tensor(imgsC_val[valIdx_i]).cuda().float()
        valInput_ID=plabels_val[valIdx_i].cuda()
        valIdx_i=torch.tensor(valIdx_i)

        reconShared_dna_l,recon_dna_l= modelcnn_dna(valtarget_dna,valInput_ID)
        reconShared_protein_l,recon_protein_l= modelcnn_protein(valtarget_protein,valInput_ID)

        recon_dna= modelcnn_dna_dec(torch.cat((reconShared_dna_l,recon_dna_l),dim=1),pIDemb_dec(valInput_ID))
        reconShared_dna=modelcnn_dnaShared_dec(reconShared_dna_l,pIDemb_dec(valInput_ID))
        recon_protein= modelcnn_protein_dec(torch.cat((reconShared_protein_l,recon_protein_l),dim=1),pIDemb_dec(valInput_ID))
        reconShared_protein=modelcnn_pShared_dec(reconShared_protein_l,pIDemb_dec(valInput_ID))


        loss_x_val_protein=loss_imgRecon(recon_protein, valtarget_protein)
        loss_xShared_val_protein=loss_imgRecon(reconShared_protein,valtarget_protein)
        loss_x_val_dna=loss_imgRecon(recon_dna, valtarget_dna)
        loss_xShared_val_dna=loss_imgRecon(reconShared_dna,valtarget_dna)

        loss=loss_x_val_protein+loss_xShared_val_protein+loss_x_val_dna+loss_xShared_val_dna

        loss_x_val_all_dna+=loss_x_val_dna.item()
        loss_x_valShared_all_dna+=loss_xShared_val_dna.item()
        loss_x_val_all_protein+=loss_x_val_protein.item()
        loss_x_valShared_all_protein+=loss_xShared_val_protein.item()
        loss_val_all+=loss.item()
        
        for i in range(10):
            print(i)
            print(proteinNames_val[valIdx_i][i])
            plt.imshow(percentileNorm(valtarget_protein[i][0].cpu().detach().numpy()))
            plt.show()
            plt.imshow(percentileNorm(sigmoid(reconShared_protein[i][0]).cpu().detach().numpy()))
            plt.show()
            plt.imshow(percentileNorm(sigmoid(recon_protein[i][0]).cpu().detach().numpy()))
            plt.show()
            print(loss_imgRecon(valtarget_protein[i][0],reconShared_protein[i][0]).item())
            print(loss_imgRecon(valtarget_protein[i][0],recon_protein[i][0]).item())
            
            plt.imshow(modeSub(valtarget_dna[i][0].cpu().detach().numpy()))
            plt.show()
            plt.imshow(modeSub(sigmoid(reconShared_dna[i][0]).cpu().detach().numpy()))
            plt.show()
            plt.imshow(modeSub(sigmoid(recon_dna[i][0]).cpu().detach().numpy()))
            plt.show()
            print(loss_imgRecon(reconShared_dna[i][0],valtarget_dna[i][0]).item())
            print(loss_imgRecon(recon_dna[i][0],valtarget_dna[i][0]).item())

    loss_x_val_all_dna=loss_x_val_all_dna/nvalBatches
    loss_x_valShared_all_dna=loss_x_valShared_all_dna/nvalBatches
    loss_x_val_all_protein=loss_x_val_all_protein/nvalBatches
    loss_x_valShared_all_protein=loss_x_valShared_all_protein/nvalBatches
    loss_val_all=loss_val_all/nvalBatches

In [ ]:
#get plotting index

plottingIdx=np.array([]).astype(int)

nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
for i in range(nvalBatches):
    valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]

    plottingIdx=np.concatenate((plottingIdx,valIdx_i[:10]))

        


In [ ]:
plottingIdx

In [ ]:
pnames

In [ ]:
#plotting prediction of  all proteins
with torch.no_grad():
    modelcnn_dna.eval()
    modelcnn_protein.eval()
    modelcnn_dna.cuda()
    modelcnn_protein.cuda()


    for i in range(plottingIdx.size):
        print(i)
        print('input img',proteinNames_val[plottingIdx][i])
        
        
        valtarget_protein=torch.tensor(imgsP_val[[plottingIdx[i]]]).cuda().float()
        valtarget_dna=torch.tensor(imgsC_val[[plottingIdx[i]]]).cuda().float()
        valInput_ID_orig=plabels_val[[plottingIdx[i]]].cuda()
        valIdx_i=torch.tensor([plottingIdx[i]])
        valAllProteins=imgsP_val_all[plottingIdx[i]]
        valAllProteins_names=imgsP_val_all_names[plottingIdx[i]]
        
        plt.imshow(modeSub(valtarget_dna[0][0].cpu().detach().numpy()))
        plt.show()
#         plt.imshow(percentileNorm(valtarget_protein[0][0].cpu().detach().numpy()))
#         plt.show()
        for pidx in range(3):
            if valAllProteins_names[pidx]=='None':
                continue
            print('True ',valAllProteins_names[pidx])
            plt.imshow(percentileNorm(valAllProteins[pidx]))
            plt.show()
        
        for pidx in range(pnames.size):
            print(pnames[pidx])
            valInput_ID=torch.tensor([pidx]).cuda()
            
            reconShared_dna_l,recon_dna_l= modelcnn_dna(valtarget_dna,valInput_ID_orig)
            reconShared_protein_l,recon_protein_l= modelcnn_protein(valtarget_protein,valInput_ID_orig)

            recon_protein= modelcnn_protein_dec(torch.cat((reconShared_dna_l,recon_protein_l),dim=1),pIDemb_dec(valInput_ID))
            reconShared_protein=modelcnn_pShared_dec(reconShared_dna_l,pIDemb_dec(valInput_ID))

            plt.imshow(percentileNorm(sigmoid(reconShared_protein[0][0]).cpu().detach().numpy()))
            plt.show()
            plt.imshow(percentileNorm(sigmoid(recon_protein[0][0]).cpu().detach().numpy()))
            plt.show()
            plt.imshow(modeSub(sigmoid(reconShared_protein[0][0]).cpu().detach().numpy()))
            plt.show()

            



In [ ]:
loss_x_noReduction=torch.nn.MSELoss(reduction='sum')

In [ ]:
imgsC_val_allProt={}
imgsP_val_allProt={}
imgsP_val_allProt_input={}
imgNames_val_allProt={}
pID_val_allProt={}
conditions_val_allProt={}
proteinNames_val_allProt={}
for condition_i in conditions:
    print(condition_i)
    segDir_i=os.path.join(segDir,condition_i)
    imgDir_i=os.path.join(imgDir,condition_i)
    for stain in os.listdir(segDir_i):
        print(stain)
        segDir_i_stain=os.path.join(segDir_i,stain)
        imgDir_i_stain=os.path.join(imgDir_i,stain)
        
        segPID2name={}
        for pID_dir in os.listdir(segDir_i_stain):
            pID=pID_dir.split('_')
            segPID2name[pID[0]]=pID_dir
        imgPID2name={}
        for pID_dir in os.listdir(imgDir_i_stain):
            pID=pID_dir.split('_')
            imgPID2name[pID[0]]=pID_dir
        for pID in segPID2name.keys():
            if condition_i=='meningioma' and stain=='dapi_gh2ax_lamin_cd3' and pID=='P33': #skipping incorrect images
                continue
            if pID not in holdOutSamples:
                continue
            print(pID)
            if pID not in imgPID2name:
                print('img not found '+pID)
                continue
            imgDir_i_stain_p=os.path.join(imgDir_i_stain,imgPID2name[pID])
            segDir_i_stain_p=os.path.join(segDir_i_stain,segPID2name[pID])
            
            with open(os.path.join(imgDir_i_stain_p,savename+'_imgNames'), 'rb') as output:
                imgNames=pickle.load(output)
            with open(os.path.join(imgDir_i_stain_p,savename+'_img'), 'rb') as output:
                img=pickle.load(output)
                
#             imgP=np.zeros((img.shape[0],1,img.shape[2],img.shape[3]))
#             proteinNames_val_curr=np.array([])
            stain_list=stain.split('_')
            nImgPerStain=int(img.shape[0]/(len(stain_list)-1))
            np.random.seed(3)
            allIdx_all=np.arange(img.shape[0])
            np.random.shuffle(allIdx_all)
            for s in range(1,len(stain_list)):
                s_start=(s-1)*nImgPerStain
                if s==len(stain_list)-1:
                    s_end=img.shape[0]
                else:
                    s_end=s*nImgPerStain
                proteinNames_val_curr=np.repeat(stain_list[s],s_end-s_start)
                imgP=img[allIdx_all[s_start:s_end],s].reshape(s_end-s_start,1,img.shape[2],img.shape[3])

                for sother in range(1,len(stain_list)):
                    if sother==s:
                        continue
                    if stain_list[sother] not in imgsP_val_allProt.keys():
                        pID_val_allProt[stain_list[sother]]=np.repeat(pID,s_end-s_start)
                        imgsC_val_allProt[stain_list[sother]]=img[allIdx_all[s_start:s_end],[0]].reshape(s_end-s_start,1,imgsC_val.shape[2],imgsC_val.shape[3])
                        imgNames_val_allProt[stain_list[sother]]=imgNames[allIdx_all[s_start:s_end]]
                        imgsP_val_allProt[stain_list[sother]]=img[allIdx_all[s_start:s_end],[sother]].reshape(s_end-s_start,1,imgsC_val.shape[2],imgsC_val.shape[3])
                        conditions_val_allProt[stain_list[sother]]=np.repeat(condition_i,s_end-s_start)
                        proteinNames_val_allProt[stain_list[sother]]=proteinNames_val_curr
                        imgsP_val_allProt_input[stain_list[sother]]=imgP
                    else:
                        pID_val_allProt[stain_list[sother]]=np.concatenate((pID_val_allProt[stain_list[sother]],np.repeat(pID,s_end-s_start)))
                        imgsC_val_allProt[stain_list[sother]]=np.concatenate((imgsC_val_allProt[stain_list[sother]],img[allIdx_all[s_start:s_end],[0]].reshape(s_end-s_start,1,imgsC_val.shape[2],imgsC_val.shape[3])),axis=0)
                        imgNames_val_allProt[stain_list[sother]]=np.concatenate((imgNames_val_allProt[stain_list[sother]],imgNames[allIdx_all[s_start:s_end]]))
                        imgsP_val_allProt[stain_list[sother]]=np.concatenate((imgsP_val_allProt[stain_list[sother]],img[allIdx_all[s_start:s_end],[sother]].reshape(s_end-s_start,1,imgsC_val.shape[2],imgsC_val.shape[3])),axis=0)
                        conditions_val_allProt[stain_list[sother]]=np.concatenate((conditions_val_allProt[stain_list[sother]],np.repeat(condition_i,s_end-s_start)))
                        proteinNames_val_allProt[stain_list[sother]]=np.concatenate((proteinNames_val_allProt[stain_list[sother]],proteinNames_val_curr))
                        imgsP_val_allProt_input[stain_list[sother]]=np.concatenate((imgsP_val_allProt_input[stain_list[sother]],imgP),axis=0)


In [ ]:
imgsC_val.shape

In [ ]:
imgsP_val_allProt[pnames[pidx]].shape

In [ ]:
#prediction loss of all proteins - l1 + thresholding
loss_l1=torch.nn.L1Loss()
with torch.no_grad():
    modelcnn_dna.eval()
    modelcnn_protein.eval()

    

    for pidx in range(pnames.size):
        print(pnames[pidx])
        
        plabels_orig=torch.zeros(proteinNames_val_allProt[pnames[pidx]].size,dtype=int)
        for pidx_label in range(pnames.size):
            plabels_orig[proteinNames_val_allProt[pnames[pidx]]==pnames[pidx_label]]=pidx_label
        
        valInput_ID_single=torch.tensor([pidx]).cuda()
        
        valIdx_p=np.arange(imgsP_val_allProt[pnames[pidx]].shape[0])
        loss_x_valShared_all_protein=0
        loss_x_val_all_protein=0
        nvalBatches=int(np.ceil(valIdx_p.shape[0]/batchsize))
        for i in range(nvalBatches):
            valIdx_i=valIdx_p[i*batchsize:min((i+1)*batchsize,valIdx_p.shape[0])]
            valtarget_protein=torch.tensor(imgsP_val_allProt_input[pnames[pidx]][valIdx_i]).cuda().float()
            valtarget_protein_pred=torch.tensor(imgsP_val_allProt[pnames[pidx]][valIdx_i]).cuda().float()
            valtarget_dna=torch.tensor(imgsC_val_allProt[pnames[pidx]][valIdx_i]).cuda().float()
            valInput_ID=torch.repeat_interleave(valInput_ID_single,valIdx_i.shape[0]).cuda()
            valInput_ID_orig=plabels_orig[valIdx_i].cuda()
            valIdx_i=torch.tensor(valIdx_i)

            reconShared_dna_l,recon_dna_l= modelcnn_dna(valtarget_dna,valInput_ID_orig)
            reconShared_protein_l,recon_protein_l= modelcnn_protein(valtarget_protein,valInput_ID_orig)

            reconShared_protein=modelcnn_pShared_dec(reconShared_dna_l,pIDemb_dec(valInput_ID))
            recon_protein= modelcnn_protein_dec(torch.cat((reconShared_dna_l,recon_protein_l),dim=1),pIDemb_dec(valInput_ID))



            loss_xShared_val_protein=loss_l1(modeSub_torch(sigmoid(reconShared_protein)),valtarget_protein_pred)
            loss_x_val_protein=loss_l1(modeSub_torch(sigmoid(recon_protein)),valtarget_protein_pred)

            loss_x_valShared_all_protein+=loss_xShared_val_protein.item()
            loss_x_val_all_protein+=loss_x_val_protein.item()


        loss_x_valShared_all_protein=loss_x_valShared_all_protein/nvalBatches
        loss_x_val_all_protein=loss_x_val_all_protein/nvalBatches
        print(loss_x_valShared_all_protein)
        print(loss_x_val_all_protein)

In [ ]:
#l1
loss_x_noReduction=torch.nn.L1Loss(reduction='sum')
with torch.no_grad():
    modelcnn_dna.eval()
    modelcnn_protein.eval()
    modelcnn_dna.cuda()
    modelcnn_protein.cuda()



    loss_val_all=0
    loss_x_val_all_dna=0
    loss_x_valShared_all_dna=0
    loss_x_val_all_protein=0
    loss_x_valShared_all_protein=0
    loss_shared={}
    loss_shared['gh2ax']=0
    loss_shared['cd16']=0
    loss_shared['cd3']=0
    loss_shared['cd4']=0
    loss_shared['cd8']=0
    loss_shared['lamin']=0
    loss_full={}
    loss_full['gh2ax']=0
    loss_full['cd16']=0
    loss_full['cd3']=0
    loss_full['cd4']=0
    loss_full['cd8']=0
    loss_full['lamin']=0
    nvalBatches=int(np.ceil(valIdx.shape[0]/batchsize))
    for i in range(nvalBatches):
        valIdx_i=valIdx[i*batchsize:min((i+1)*batchsize,valIdx.shape[0])]
        valtarget_protein=torch.tensor(imgsP_val[valIdx_i]).cuda().float()
        valtarget_dna=torch.tensor(imgsC_val[valIdx_i]).cuda().float()
        valInput_ID=plabels_val[valIdx_i].cuda()
        valIdx_i=torch.tensor(valIdx_i)

        reconShared_dna_l,recon_dna_l= modelcnn_dna(valtarget_dna,valInput_ID)
        reconShared_protein_l,recon_protein_l= modelcnn_protein(valtarget_protein,valInput_ID)

        recon_dna= modelcnn_dna_dec(torch.cat((reconShared_dna_l,recon_dna_l),dim=1),pIDemb_dec(valInput_ID))
        reconShared_dna=modelcnn_dnaShared_dec(reconShared_dna_l,pIDemb_dec(valInput_ID))
        recon_protein= modelcnn_protein_dec(torch.cat((reconShared_protein_l,recon_protein_l),dim=1),pIDemb_dec(valInput_ID))
        reconShared_protein=modelcnn_pShared_dec(reconShared_protein_l,pIDemb_dec(valInput_ID))


        loss_x_val_protein=loss_x(sigmoid(recon_protein), valtarget_protein)
        loss_xShared_val_protein=loss_x(sigmoid(reconShared_protein),valtarget_protein)
        loss_x_val_dna=loss_x(sigmoid(recon_dna), valtarget_dna)
        loss_xShared_val_dna=loss_x(sigmoid(reconShared_dna),valtarget_dna)

        loss=loss_x_val_protein+loss_xShared_val_protein+loss_x_val_dna+loss_xShared_val_dna

        loss_x_val_all_dna+=loss_x_val_dna.item()
        loss_x_valShared_all_dna+=loss_xShared_val_dna.item()
        loss_x_val_all_protein+=loss_x_val_protein.item()
        loss_x_valShared_all_protein+=loss_xShared_val_protein.item()
        loss_val_all+=loss.item()
        
        for p in np.unique(proteinNames_val[valIdx_i]):
            loss_full[p]+=loss_x_noReduction(modeSub_torch(sigmoid(recon_protein[proteinNames_val[valIdx_i]==p])), valtarget_protein[proteinNames_val[valIdx_i]==p]).item()/(valtarget_dna.shape[2]*valtarget_dna.shape[3])
            loss_shared[p]+=loss_x_noReduction(modeSub_torch(sigmoid(reconShared_protein[proteinNames_val[valIdx_i]==p])),valtarget_protein[proteinNames_val[valIdx_i]==p]).item()/(valtarget_dna.shape[2]*valtarget_dna.shape[3])
#             loss_full[p]+=loss_x_noReduction(sigmoid(recon_protein[proteinNames_val[valIdx_i]==p]), valtarget_protein[proteinNames_val[valIdx_i]==p]).item()/(valtarget_dna.shape[2]*valtarget_dna.shape[3])
#             loss_shared[p]+=loss_x_noReduction(sigmoid(reconShared_protein[proteinNames_val[valIdx_i]==p]),valtarget_protein[proteinNames_val[valIdx_i]==p]).item()/(valtarget_dna.shape[2]*valtarget_dna.shape[3])


    loss_x_val_all_dna=loss_x_val_all_dna/nvalBatches
    loss_x_valShared_all_dna=loss_x_valShared_all_dna/nvalBatches
    loss_x_val_all_protein=loss_x_val_all_protein/nvalBatches
    loss_x_valShared_all_protein=loss_x_valShared_all_protein/nvalBatches
    loss_val_all=loss_val_all/nvalBatches
    
    for p in np.unique(proteinNames_val):
        print(p)
        loss_shared[p]=loss_shared[p]/np.sum(proteinNames_val==p)
        loss_full[p]=loss_full[p]/np.sum(proteinNames_val==p)
        print('shared',loss_shared[p])
        print('full',loss_full[p])

In [ ]:
model_losses=pd.read_csv('l1_thresh_recon.csv',index_col=0)

In [ ]:
x = np.arange(7)  # the label locations
width = 0.12  
multiplier = 0

fig, ax = plt.subplots(layout='constrained',figsize=(7,5))

colors=['deeppink','lightsalmon','slategrey']
for i in range(3):
    offset = width * multiplier
    rects = ax.bar(x + offset, model_losses.iloc[i], width, label=model_losses.index[i],color=colors[i])
    multiplier += 1
ax.set_yscale('log')
ax.set_xticks(x + width, model_losses.columns)
ax.legend(loc='upper right')
ax.hlines(y=1,xmin=-0.3,xmax=6.5,colors='green',linestyles='dashed')

In [ ]:
model_losses_pred=pd.read_csv('l1_thresh_predloss_fc.csv',index_col=0)

In [ ]:
x = np.arange(6)  # the label locations
width = 0.12  
multiplier = 0

fig, ax = plt.subplots(layout='constrained',figsize=(8,3))

colors=['deeppink','lightsalmon','slategrey','lightskyblue']
for i in range(3):
    offset = width * multiplier
    rects = ax.bar(x + offset, model_losses_pred.iloc[i], width, label=model_losses_pred.index[i],color=colors[i])
    multiplier += 1
ax.set_yscale('log')
ax.set_xticks(x + width, model_losses_pred.columns)
ax.set_ylim(top=2)
ax.legend(loc='upper left', bbox_to_anchor=(1, 1))
ax.hlines(y=1,xmin=-0.2,xmax=5.5,colors='green',linestyles='dashed')